# Data Preprocessing Based on Saved Files from NB 01 and 01b

Import necessary packages

In [592]:
import glob
import json
import os
import pandas as pd
import warnings
import sqlite3


## Cleaning Zillow home data

### Import data and combine into df

In [593]:

warnings.filterwarnings("ignore")
DATA_FOLDER = '../data/raw'
#making a list of file names to easily access them
zillow_files = [os.path.join(DATA_FOLDER, 'zillow',  file) 
             for file in os.listdir(DATA_FOLDER + "/zillow") 
             if file.endswith('.json')]

combined_data = []
for file in zillow_files:
  #found a stackoverflow how to read json files line by line as dictionaries
  with open(file, 'r') as f:
    for line in f:
      combined_data.append(json.loads(line))
    
    
#converting the list of dictionaries to a pandas dataframe
combined_df = pd.DataFrame.from_dict(combined_data)



### Drop unnecessary columns, convert to proper dtypes

In [594]:
combined_cleaned = combined_df[['id', 'price', 'address', 
    'addressZipcode', 'addressState', 'beds', 'baths', 'area', 
    'zestimate', 'brokerName']]
#free up space
del combined_df


I only want houses which are real (have at least 1 bedroom)

In [595]:
combined_real = combined_cleaned[combined_cleaned['beds'] > 0]
# display(combined_real.info())
# display(combined_real.head())

Now all houses have non-null beds, baths, etc
Next is to strip the $ sign from price, add a town column, and convert to proper data types

In [596]:
#convert the price to float and remove the $ and , and + signs.
#some houses had a + sign at the end of the price, but I took the price without the sign
combined_real['price']  = combined_real['price'].str.replace('$', '').str.replace(',','').str.replace('+', '')
combined_real['price'] = combined_real['price'].astype(float)

Bedroom can only be an integer, so will cast it as an int16. ID is also an int. Add the town column for maybe more insights down the line

In [597]:
combined_real['beds'] = combined_real['beds'].astype('int16')
combined_real['id'] = combined_real['id'].astype('int64')
combined_real['town'] = combined_real['address'].str.split(',').str[1]
combined_real.drop_duplicates(subset='address', inplace=True)


## Cleaning Zip Data

### Load in the data

In [598]:
zip_data = json.load(open('../data/raw/zip_info.json'))
zip_info = [x['highlights'] for x in zip_data]
zip_df = pd.DataFrame(zip_info)
zip_df['zip_codes'] = [x['selectedProfile']['label'] for x in zip_data]
display(zip_df.head())



,0,1,2,3,4,5,6,7,8,zip_codes
0,"{'format': 'number', 'value': '4228', 'topic':...","{'format': 'dollar', 'value': '170379', 'topic...","{'format': 'percent', 'value': '75.8', 'topic'...","{'format': 'percent', 'value': '51.7', 'topic'...","{'format': 'number', 'value': '2321', 'topic':...","{'format': 'percent', 'value': '1.9', 'topic':...","{'format': 'number', 'value': '1087', 'topic':...","{'format': 'number', 'value': '2002', 'topic':...","{'format': 'number', 'value': '256', 'topic': ...",ZCTA5 02108
1,"{'format': 'number', 'value': '3921', 'topic':...","{'format': 'dollar', 'value': '162179', 'topic...","{'format': 'percent', 'value': '81.5', 'topic'...","{'format': 'percent', 'value': '77.5', 'topic'...","{'format': 'number', 'value': '2639', 'topic':...","{'format': 'percent', 'value': '1.5', 'topic':...","{'format': 'number', 'value': '1340', 'topic':...","{'format': 'number', 'value': '1993', 'topic':...","{'format': 'number', 'value': '205', 'topic': ...",ZCTA5 02109
2,"{'format': 'number', 'value': '2921', 'topic':...","{'format': 'dollar', 'value': '126157', 'topic...","{'format': 'percent', 'value': '85.0', 'topic'...","{'format': 'percent', 'value': '55.2', 'topic'...","{'format': 'number', 'value': '1754', 'topic':...","{'format': 'percent', 'value': '0.7', 'topic':...","{'format': 'number', 'value': '1917', 'topic':...","{'format': 'number', 'value': '1312', 'topic':...","{'format': 'number', 'value': '132', 'topic': ...",ZCTA5 02110
3,"{'format': 'number', 'value': '9716', 'topic':...","{'format': 'dollar', 'value': '72321', 'topic'...","{'format': 'percent', 'value': '61.5', 'topic'...","{'format': 'percent', 'value': '55.9', 'topic'...","{'format': 'number', 'value': '4827', 'topic':...","{'format': 'percent', 'value': '0.9', 'topic':...","{'format': 'number', 'value': '963', 'topic': ...","{'format': 'number', 'value': '3746', 'topic':...","{'format': 'number', 'value': '745', 'topic': ...",ZCTA5 02111
4,"{'format': 'number', 'value': '29073', 'topic'...","{'format': 'dollar', 'value': '74155', 'topic'...","{'format': 'percent', 'value': '54.8', 'topic'...","{'format': 'percent', 'value': '57.2', 'topic'...","{'format': 'number', 'value': '15080', 'topic'...","{'format': 'percent', 'value': '2.8', 'topic':...","{'format': 'number', 'value': '894', 'topic': ...","{'format': 'number', 'value': '14133', 'topic'...","{'format': 'number', 'value': '5191', 'topic':...",ZCTA5 02118


In [599]:
columns_temp = [x['label'] for x in zip_info[0]]
columns_final = [x.replace(' ', '_').replace('\'','').lower() for x in columns_temp]
columns_final.append('zcta')
# print(columns_final)
zip_df.columns = columns_final
#extract value from all columns since they were dictionaries with other information tags
for name in zip_df.columns:
  if name == 'zcta':
    zip_df[name] = zip_df[name].apply(lambda x: x[5:])
    continue
  # print("name is ", name)
  zip_df[name] = zip_df[name].apply(lambda x: x['value'] if type(x) == dict else None)
# display(zip_df.info())


### Convert zip df to proper type

Now, I want to remove the zip codes with 0 households (which also removes '-' from other columns in same row)

In [600]:

zip_df.dropna(subset=['total_households'], inplace=True)
zip_df = zip_df[zip_df['total_households'].astype(int) > 0]

# display(zip_df)

In [601]:
%load_ext sql
%config SqlMagic.autocommit=True # for engines that do not support autommit

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [609]:
DATA_FOLDER_P = os.path.join('../data')

In [610]:
conn = sqlite3.connect(os.path.join(DATA_FOLDER_P, './home_prices.db'))
# # conn.commit()
# # conn.close()

In [604]:
# display(combined_real.head())
combined_real.to_sql('zillow', conn, if_exists='replace', index=False)
zip_df.to_sql('zip_info', conn, if_exists='replace', index=False)

15

Make the schema for the zillow data

In [611]:
%%sql homes
SELECT name FROM sqlite_master WHERE type='table';




id,price,address,addressZipcode,addressState,beds,baths,area,zestimate,brokerName,town
59150059,1595000.0,"16 Atlantic St, South Boston, MA 02127",02127,MA,5,3.0,2300.0,1538400.0,Grove Property Group,South Boston
2063017667,899000.0,"330 1/2 East St #1, Boston, MA 02127",02127,MA,4,2.0,1662.0,None,Gibson Sotheby's International Realty,Boston
63712077,850000.0,"534-538 Commonwealth Ave UNIT 7D, Boston, MA 02215",02215,MA,2,2.0,864.0,800100.0,Cornerstone Real Estate,Boston
318227332,939000.0,"183 D St UNIT 4, South Boston, MA 02127",02127,MA,2,2.0,1037.0,927800.0,Skyline Realty,South Boston
190014785,899000.0,"22 Florian St, Roslindale, MA 02131",02131,MA,6,2.0,1959.0,872500.0,Here Realty LLC,Roslindale


In [612]:
conn.close()